<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Calculate Windspeed and Direction for Each Model and Timestep</h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 12/11/23
<br>
<br>    
<br>
Takes models/variables downloaded and calculates the windspeed and direction. File_concat_mod_functions.ipynb imports the function that computes the 24HR AVG, MIN, MAX outputs. 
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

## Import needed libraries, etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from dask.distributed import Client, LocalCluster
import dask.array as da
import os
import glob
from metpy.units import units
import math
import metpy

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None

## Establish a dask client. This is a lot of data.

In [ ]:
Cluster = LocalCluster(n_workers = 8, threads_per_worker=4, memory_limit='30GB',  processes=True)
#Cluster = LocalCluster()

In [ ]:
client = Client(Cluster)
client

### The integral notebook of functions to run

In [ ]:
%run File_concat_mod_functions.ipynb

the main function to run is:

    return_concat(model, variable, prototype = None)

Where the prototype is only needed for the UFS_S2S model choice. 

# These are the available options the functions are designed for

Not all variables will be available from all models but most are. 

In [ ]:
model_options = ['CONUS404', 'ERA5', 'HRRR', 'NAM', 'NARR', 'NCEP', 'UFS_S2S']
variable_options = ['PBL', 'CAPE', 'SOILM', 'WIND', 'PRECIP', 'TEMP', 'RH']

In [ ]:
output_dir = 'database_files'

## CONUS404

In [ ]:
%%time

conus404_wind = return_concat('CONUS404', 'WIND')

In [ ]:
conus404_wind

In [ ]:
w_speed_conus404 = metpy.calc.wind_speed(conus404_wind.U10 * units.meter_per_second, conus404_wind.V10 * units.meter_per_second)
w_speed_conus404 = w_speed_conus404.metpy.dequantify().to_dataset(name = 'wspeed')

In [ ]:
wind_direction_conus404 = (270 - np.degrees(np.arctan2(conus404_wind.V10, conus404_wind.U10))) % 360
wind_direction_conus404 = wind_direction_conus404.to_dataset(name = 'wdir')

Get min, max, absolute for the new variable

In [ ]:
%%time
resampler_regular_vars('wspeed', w_speed_conus404, output_dir, 'CONUS404')

In [ ]:
%%time
resampler_regular_vars('wdir', wind_direction_conus404, output_dir, 'CONUS404')

## ERA5
expected timesteps = 1460

In [ ]:
%%time

era_wind = return_concat('ERA5', 'WIND')

In [ ]:
w_speed_era = metpy.calc.wind_speed(era_wind.u10 * units.meter_per_second, era_wind.v10 * units.meter_per_second)
w_speed_era = w_speed_era.metpy.dequantify().to_dataset(name = 'wspeed')

In [ ]:
wind_direction_era = (270 - np.degrees(np.arctan2(era_wind.v10, era_wind.u10))) % 360
wind_direction_era = wind_direction_era.to_dataset(name = 'wdir')

Get min, max, absolute for the new variable

In [ ]:
%%time
resampler_regular_vars('wspeed', w_speed_era, output_dir, 'ERA5')

In [ ]:
resampler_regular_vars('wdir', wind_direction_era, output_dir, 'ERA5')

## HRRR
expected timesteps = 1460

In [ ]:
model = 'HRRR'

In [ ]:
model_list = []
parent_dir = f'{output_dir}/{model}'

In [ ]:
vpd_variable_options = ['WIND']
model_vars = []

In [ ]:
for v in vpd_variable_options:
    v = get_model_var(model, v)
    model_vars.append(v)
    for vvv in v:
        model_list.append(sorted(glob.glob(os.path.join(parent_dir, f'{vvv}_{get_filename(model)}_Abs_*.nc'))))


In [ ]:
model_vars

In [ ]:
model_list = model_list[0:2]

In [ ]:
if len(np.unique([len(i) for i in model_list])) >1:
    print('the number of years for each variable are not the same')

In [ ]:
model_list

In [ ]:
%%time

for u,v in zip(model_list[0], model_list[1]):
    if int(u[-7:-3]) != int(v[-7:-3]):
        print('the years for each variable are not aligned, rh')
    else:
        print(u, v)

        u10 = xr.open_dataset(u)
        v10 = xr.open_dataset(v)

        print('calculating wind speed')
        w_speed_hrrr = metpy.calc.wind_speed(u10.u10 * units.meter_per_second, v10.v10 * units.meter_per_second)
        w_speed_hrrr = w_speed_hrrr.metpy.dequantify().to_dataset(name = 'wspeed')
        
        print('calculating wind dir')
        wind_direction_hrrr = (270 - np.degrees(np.arctan2(v10.v10, u10.u10))) % 360
        wind_direction_hrrr = wind_direction_hrrr.to_dataset(name = 'wdir')

        print('resampling')
        resampler_regular_vars('wspeed', w_speed_hrrr, output_dir, 'HRRR')
        resampler_regular_vars('wdir', wind_direction_hrrr, output_dir, 'HRRR')
        

## NAM
expected timesteps = 1460

In [ ]:
model = 'NAM'

In [ ]:
model_list = []
parent_dir = f'{output_dir}/{model}'

In [ ]:
vpd_variable_options = ['WIND']
model_vars = []

In [ ]:
for v in vpd_variable_options:
    v = get_model_var(model, v)
    model_vars.append(v)
    for vvv in v:
        model_list.append(sorted(glob.glob(os.path.join(parent_dir, f'{vvv}_{get_filename(model)}_Abs_*2012.nc'))))


In [ ]:
model_vars

In [ ]:
model_list = model_list[0:2]

In [ ]:
if len(np.unique([len(i) for i in model_list])) >1:
    print('the number of years for each variable are not the same')

In [ ]:
model_list

In [ ]:
%%time

for u,v in zip(model_list[0], model_list[1]):
    if int(u[-7:-3]) != int(v[-7:-3]):
        print('the years for each variable are not aligned, rh')
    else:
        print(u, v)

        u10 = xr.open_dataset(u)
        v10 = xr.open_dataset(v)

        print('calculating wind speed')
        w_speed_nam = metpy.calc.wind_speed(u10.u10 * units.meter_per_second, v10.v10 * units.meter_per_second)
        w_speed_nam = w_speed_nam.metpy.dequantify().to_dataset(name = 'wspeed')
        
        print('calculating wind dir')
        wind_direction_nam = (270 - np.degrees(np.arctan2(v10.v10, u10.u10))) % 360
        wind_direction_nam = wind_direction_nam.to_dataset(name = 'wdir')

        print('resampling')
        resampler_regular_vars('wspeed', w_speed_nam, output_dir, 'NAM')
        resampler_regular_vars('wdir', wind_direction_nam, output_dir, 'NAM')
        

## NARR
expected timesteps = 1460

In [ ]:
%%time

narr_wind = return_concat('NARR', 'WIND')

In [ ]:
narr_wind

In [ ]:
w_speed_narr = metpy.calc.wind_speed(narr_wind['u-component_of_wind_height_above_ground'] * units.meter_per_second, narr_wind['v-component_of_wind_height_above_ground'] * units.meter_per_second)
w_speed_narr = w_speed_narr.metpy.dequantify().to_dataset(name = 'wspeed')

In [ ]:
wind_direction_narr = (270 - np.degrees(np.arctan2(narr_wind['v-component_of_wind_height_above_ground'], narr_wind['u-component_of_wind_height_above_ground']))) % 360
wind_direction_narr = wind_direction_narr.to_dataset(name = 'wdir')

Get min, max, absolute for the new variable

In [ ]:
%%time
resampler_regular_vars('wspeed', w_speed_narr, output_dir, 'NARR')

In [ ]:
%%time
resampler_regular_vars('wdir', wind_direction_narr, output_dir, 'NARR')

## NCEP
expected timesteps = 1460

In [ ]:
%%time

ncep_wind = return_concat('NCEP', 'WIND')

In [ ]:
ncep_wind

In [ ]:
w_speed_ncep = metpy.calc.wind_speed(ncep_wind.uwnd * units.meter_per_second, ncep_wind.vwnd * units.meter_per_second)
w_speed_ncep = w_speed_ncep.metpy.dequantify().to_dataset(name = 'wspeed')

In [ ]:
wind_direction_ncep = (270 - np.degrees(np.arctan2(ncep_wind.vwnd, ncep_wind.uwnd))) % 360
wind_direction_ncep = wind_direction_ncep.to_dataset(name = 'wdir')

Get min, max, absolute for the new variable

In [ ]:
%%time
resampler_regular_vars('wspeed', w_speed_ncep, output_dir, 'NCEP')

In [ ]:
%%time
resampler_regular_vars('wdir', wind_direction_ncep, output_dir, 'NCEP')